In [1]:
import pandas as pd

# Load GTFS core files
calendar = pd.read_csv("./data/calendar.txt")
# fix dates
calendar['start_date'] = pd.to_datetime(calendar['start_date'], format='%Y%m%d')
calendar['end_date'] = pd.to_datetime(calendar['end_date'], format='%Y%m%d')

calendar_dates = pd.read_csv("./data/calendar_dates.txt")
calendar_dates['date'] = pd.to_datetime(calendar_dates['date'], format='%Y%m%d')

trips = pd.read_csv("./data/trips.txt")
stop_times = pd.read_csv("./data/stop_times.txt")
stops = pd.read_csv("./data/stops.txt")
routes = pd.read_csv("./data/routes.txt")
agency = pd.read_csv("./data/agency.txt")

print("Data loaded successfully.")

Data loaded successfully.


In [2]:
def getArrivalsAndDepartures(stopId, date) -> pd.DataFrame:
    """
    Get arrivals and departures for a specific stop on a given date.
    
    :param stopId: The ID of the stop.
    :param date: The date in YYYY-MM-DD format.
    :return: DataFrame with arrivals and departures.
    """
    # Ensure date is datetime object
    date = pd.to_datetime(date)

    # Convert date to required format for matching (YYYYMMDD as int)
    date_str = date.strftime('%Y%m%d')
    date_int = int(date_str)

    # Day name as per calendar.txt columns: monday, tuesday, ...
    day_name = date.strftime('%A').lower()

    # Step 1: Filter calendar for date range and active on that weekday
    active_services = calendar[
        (calendar['start_date'] <= date) &
        (calendar['end_date'] >= date) &
        (calendar[day_name] == 1)
    ][['service_id']].copy()

    # Step 2: Handle exceptions from calendar_dates
    exceptions = calendar_dates[calendar_dates['date'] == date_int]

    if not exceptions.empty:
        added = exceptions[exceptions['exception_type'] == 1][['service_id']]
        removed = exceptions[exceptions['exception_type'] == 2][['service_id']]
        
        if not added.empty:
            active_services = pd.concat([active_services, added], ignore_index=True)
        
        if not removed.empty:
            active_services = active_services[~active_services['service_id'].isin(removed['service_id'])]

    active_services = active_services.drop_duplicates()
    
    print(f"Active services for {date} ({day_name}): {active_services} found.")
    # If no active service, return empty DataFrame
    if active_services.empty:
        return pd.DataFrame(columns=['trip_id', 'stop_id', 'arrival_time', 'departure_time', 'stopNumber'])

    # Step 3: Filter trips and stop_times
    trips_filtered = trips[trips['service_id'].isin(active_services['service_id'])]

    # Assign stopNumber as the index of the stop within each trip sorted by arrival_time
    stop_times_sorted = stop_times.sort_values(by=['trip_id', 'arrival_time']).copy()
    stop_times_sorted['stopNumber'] = stop_times_sorted.groupby('trip_id').cumcount()

    # Step 4: Filter for matching stopId and tripId
    stop_times_filtered = stop_times_sorted[
        (stop_times_sorted['trip_id'].isin(trips_filtered['trip_id'])) &
        (stop_times_sorted['stop_id'] == stopId)
    ]

    return stop_times_filtered[['trip_id', 'stop_id', 'arrival_time', 'departure_time', 'stopNumber']].sort_values(by='arrival_time')


In [3]:
arrivalsFor26080 = getArrivalsAndDepartures(26080, '2025-05-05')
arrivalsFor26080.to_csv('arrivalsFor26080_4may.csv', index=False)
arrivalsFor26080

Active services for 2025-05-05 00:00:00 (monday):     service_id
30        9313
31        7638
35        5884 found.


,trip_id,stop_id,arrival_time,departure_time,stopNumber


In [8]:
arrivalsFor26080 = getArrivalsAndDepartures(26080, '2025-07-31')
arrivalsFor26080.to_csv('arrivalsFor26080_2025-07-31.csv', index=False)
arrivalsFor26080

Active services for 2025-07-31 00:00:00 (thursday):    service_id
0       87621
1       86832
2       80084
3       78308
4       77519
5       64769
6       58885
8       56829
9       49148 found.


,trip_id,stop_id,arrival_time,departure_time,stopNumber
870890,724947798,26080,05:19:37,05:19:37,30
878207,724950748,26080,05:34:37,05:34:37,30
871513,724948068,26080,05:49:37,05:49:37,30
495027,686344958,26080,06:06:37,06:06:37,30
495168,686345318,26080,06:21:37,06:21:37,30
...,...,...,...,...,...
495215,686345438,26080,23:33:37,23:33:37,30
878160,724950728,26080,24:03:37,24:03:37,30
877835,724950618,26080,24:33:37,24:33:37,30
882939,724955498,26080,25:36:37,25:36:37,30


In [5]:
# get a trip that is available on 2025-05-01 but not on 2025-05-03

onMay2 = getArrivalsAndDepartures(26080, '2025-05-02')
onMay1 = getArrivalsAndDepartures(26080, '2025-05-03')

# find trips that are in onMay1 but not in onMay2
tripsNotOnMay2 = onMay1[~onMay1['trip_id'].isin(onMay2['trip_id'])]
print("Trips available on 2025-05-01 but not on 2025-05-03:")
print(tripsNotOnMay2)

Active services for 2025-05-02 00:00:00 (friday):     service_id
0        87621
10       37684
11       30003
14       27761
30        9313
31        7638
37        3429 found.
Active services for 2025-05-03 00:00:00 (saturday):     service_id
15       23519
38        2750 found.
Trips available on 2025-05-01 but not on 2025-05-03:
          trip_id  stop_id arrival_time departure_time  stopNumber
509520  686405448    26080     05:37:37       05:37:37          30
509567  686405458    26080     06:07:37       06:07:37          30
508063  686404368    26080     06:37:37       06:37:37          30
508157  686404388    26080     07:07:37       07:07:37          30
509614  686405558    26080     07:27:37       07:27:37          30
...           ...      ...          ...            ...         ...
508016  686403608    26080     24:07:37       24:07:37          30
509379  686404648    26080     24:37:37       24:37:37          30
579719  692349618    26080     25:37:37       25:37:37         

In [6]:
arrivalsFor26080 = getArrivalsAndDepartures(26080, '2025-05-03')
arrivalsFor26080.to_csv('arrivalsFor26080_3May.csv', index=False)
arrivalsFor26080

Active services for 2025-05-03 00:00:00 (saturday):     service_id
15       23519
38        2750 found.


,trip_id,stop_id,arrival_time,departure_time,stopNumber
509520,686405448,26080,05:37:37,05:37:37,30
509567,686405458,26080,06:07:37,06:07:37,30
508063,686404368,26080,06:37:37,06:37:37,30
508157,686404388,26080,07:07:37,07:07:37,30
509614,686405558,26080,07:27:37,07:27:37,30
...,...,...,...,...,...
508016,686403608,26080,24:07:37,24:07:37,30
509379,686404648,26080,24:37:37,24:37:37,30
579719,692349618,26080,25:37:37,25:37:37,30
509661,686412818,26080,26:32:37,26:32:37,30
